![JohnSnowLabs](https://nlp.johnsnowlabs.com/assets/images/logo.png)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/JohnSnowLabs/nlu/blob/master/examples/webinars_conferences_etc/multi_lingual_webinar/5_multi_lingual_sentiment_classifier_training_for_over_100_languages.ipynb.ipynb)


# Training a Sentiment Analysis Classifier with NLU 
With the [SentimentDL model](https://nlp.johnsnowlabs.com/docs/en/annotators#sentimentdl-multi-class-sentiment-analysis-annotator)  from Spark NLP you can achieve State Of the Art results on any multi class text classification problem 

This notebook showcases the following features : 

- How to train the deep learning classifier
- How to store a pipeline to disk
- How to load the pipeline from disk (Enables NLU offline mode)



# 1. Install Java 8 and NLU

In [ ]:
import os
from sklearn.metrics import classification_report
! apt-get update -qq > /dev/null   
# Install java
! apt-get install -y openjdk-8-jdk-headless -qq > /dev/null
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["PATH"] = os.environ["JAVA_HOME"] + "/bin:" + os.environ["PATH"]
! pip install  pyspark==2.4.7 
! pip install nlu > /dev/null    



import nlu

     |████████████████████████████████| 217.9MB 64kB/s 
     |████████████████████████████████| 204kB 41.1MB/s 
  Created wheel for pyspark: filename=pyspark-2.4.7-py2.py3-none-any.whl size=218279465 sha256=8f156aa79c40e63cf65136ca310fbe9fab95b918cd4b7dac9d512f3294a50f55
  Stored in directory: /root/.cache/pip/wheels/34/1f/2e/1e7460f80acf26b08dbb8c53d7ff9e07146f2a68dd5c732be5
Successfully built pyspark


# 2. Download French twitter  Sentiment dataset 
https://www.kaggle.com/hbaflast/french-twitter-sentiment-analysis
#Context

French dataset for sentiment analysis (Data translated from English to French)

In [ ]:
! wget http://ckl-it.de/wp-content/uploads/2021/02/french_tweets.csv


--2021-02-16 18:28:31--  http://ckl-it.de/wp-content/uploads/2021/02/french_tweets.csv
Resolving ckl-it.de (ckl-it.de)... 217.160.0.108, 2001:8d8:100f:f000::209
Connecting to ckl-it.de (ckl-it.de)|217.160.0.108|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10237264 (9.8M) [text/csv]
Saving to: ‘french_tweets.csv’

french_tweets.csv   100%[===================>]   9.76M  5.07MB/s    in 1.9s    

2021-02-16 18:28:33 (5.07 MB/s) - ‘french_tweets.csv’ saved [10237264/10237264]



In [ ]:
import pandas as pd
train_path = '/content/french_tweets.csv'

train_df = pd.read_csv(train_path)
# the text data to use for classification should be in a column named 'text'
columns=['text','y']
train_df = train_df[columns]
train_df = train_df.sample(frac=1).reset_index(drop=True)
train_df

,text,y
0,Nous avons apprÃ©ciÃ© la tournÃ©e en vÃ©lo du ...,positive
1,"& Quot; retournez-vous sur le chagrin brisÃ©, ...",positive
2,Le gagnant gagnant du prix a Ã©tÃ© crÃ©Ã© pour...,negative
3,Est venu maintenant de l'Ã©cole ... Ã©tait tel...,negative
4,"Dans la cuisine avec des hongrois, faisant du ...",positive
...,...,...
99995,Merci. Je ne pense pas que je serai un grand f...,positive
99996,Je pense qu'ils Ã©taient aprÃ¨s que mon petit ...,negative
99997,"Oui, j'adore les Caramellow! Seulement deux jo...",positive
99998,Je ne peux pas croire qu'elle soit Ã©veillÃ©e ...,negative


# 3. Train Deep Learning Classifier using nlu.load('train.sentiment')

You dataset label column should be named 'y' and the feature column with text data should be named 'text'

In [ ]:
# Train longer!
trainable_pipe = nlu.load('xx.embed_sentence.labse train.sentiment')
trainable_pipe['sentiment_dl'].setMaxEpochs(60)  
trainable_pipe['sentiment_dl'].setLr(0.005) 
fitted_pipe = trainable_pipe.fit(train_df.iloc[:2000])
# predict with the trainable pipeline on dataset and get predictions
preds = fitted_pipe.predict(train_df.iloc[:2000],output_level='document')

#sentence detector that is part of the pipe generates sone NaNs. lets drop them first
preds.dropna(inplace=True)
print(classification_report(preds['y'], preds['sentiment']))

preds

labse download started this may take some time.
Approximate size to download 1.7 GB
[OK!]
              precision    recall  f1-score   support

    negative       0.88      0.94      0.91       980
    positive       0.94      0.88      0.91      1020

    accuracy                           0.91      2000
   macro avg       0.91      0.91      0.91      2000
weighted avg       0.91      0.91      0.91      2000



,text,y,xx_embed_sentence_labse_embeddings,sentiment,document,sentiment_confidence
origin_index,,,,,,
0,Nous avons apprÃ©ciÃ© la tournÃ©e en vÃ©lo du ...,positive,"[0.01069789007306099, -0.002380969701334834, -...",positive,Nous avons apprÃ©ciÃ© la tournÃ©e en vÃ©lo du ...,0.999971
1,"& Quot; retournez-vous sur le chagrin brisÃ©, ...",positive,"[-0.04572410136461258, -0.04519180953502655, -...",negative,"& Quot; retournez-vous sur le chagrin brisÃ©, ...",0.999979
2,Le gagnant gagnant du prix a Ã©tÃ© crÃ©Ã© pour...,negative,"[0.008320541121065617, -0.0072850980795919895,...",negative,Le gagnant gagnant du prix a Ã©tÃ© crÃ©Ã© pour...,0.999638
3,Est venu maintenant de l'Ã©cole ... Ã©tait tel...,negative,"[-0.035592492669820786, -0.05076301470398903, ...",negative,Est venu maintenant de l'Ã©cole ... Ã©tait tel...,1.000000
4,"Dans la cuisine avec des hongrois, faisant du ...",positive,"[0.01866792142391205, -0.06534530222415924, -0...",positive,"Dans la cuisine avec des hongrois, faisant du ...",0.999910
...,...,...,...,...,...,...
1995,Vous devez suivre - transmettez-le ...,positive,"[0.0030759747605770826, -0.04513533040881157, ...",positive,Vous devez suivre - transmettez-le ...,0.999833
1996,J'allais avoir ces pantalons gris luxuriants d...,negative,"[-0.019949736073613167, 0.0064233033917844296,...",negative,J'allais avoir ces pantalons gris luxuriants d...,1.000000
1997,"Juste ""quitter"" Les gens doivent dormir mainte...",positive,"[-0.035448476672172546, -0.046279553323984146,...",positive,"Juste ""quitter"" Les gens doivent dormir mainte...",0.995777


# Test the fitted pipe on new example

# The Model understand Englsih
![en](https://www.worldometers.info/img/flags/small/tn_nz-flag.gif)

In [ ]:
fitted_pipe.predict("This was awful!")

,xx_embed_sentence_labse_embeddings,sentiment,document,sentiment_confidence
origin_index,,,,
0,"[0.034723345190286636, -0.06212149187922478, -...",negative,This was awful!,0.999652


In [ ]:
fitted_pipe.predict("This was great!")

,xx_embed_sentence_labse_embeddings,sentiment,document,sentiment_confidence
origin_index,,,,
0,"[0.03868240490555763, -0.058592554181814194, -...",positive,This was great!,0.999994


# The Model understands German
![de](https://www.worldometers.info/img/flags/small/tn_gm-flag.gif)

In [ ]:
# German for:' this movie was great!'
fitted_pipe.predict("Der Film war echt klasse!")

,xx_embed_sentence_labse_embeddings,sentiment,document,sentiment_confidence
origin_index,,,,
0,"[-0.015189268626272678, -0.04834830388426781, ...",positive,Der Film war echt klasse!,0.999732


In [ ]:
# German for: 'This movie was really boring'
fitted_pipe.predict("Der Film war echt langweilig!")

,xx_embed_sentence_labse_embeddings,sentiment,document,sentiment_confidence
origin_index,,,,
0,"[-0.013573350384831429, -0.05144454166293144, ...",negative,Der Film war echt langweilig!,0.999968


# The Model understands Chinese
![zh](https://www.worldometers.info/img/flags/small/tn_ch-flag.gif)

In [ ]:
# Chinese for: "This model was awful!"
fitted_pipe.predict("这部电影太糟糕了！")

,xx_embed_sentence_labse_embeddings,sentiment,document,sentiment_confidence
origin_index,,,,
0,"[-0.05075530335307121, -0.061598796397447586, ...",negative,这部电影太糟糕了！,1.000000


In [ ]:
# Chine for : "This move was great!"
fitted_pipe.predict("此举很棒！")


,xx_embed_sentence_labse_embeddings,sentiment,document,sentiment_confidence
origin_index,,,,
0,"[0.026034891605377197, -0.06586713343858719, -...",positive,此举很棒！,0.999995


# Model understanda Afrikaans

![af](https://www.worldometers.info/img/flags/small/tn_sf-flag.gif)



In [ ]:
# Afrikaans for 'This movie was amazing!'
fitted_pipe.predict("Hierdie film was ongelooflik!")


,xx_embed_sentence_labse_embeddings,sentiment,document,sentiment_confidence
origin_index,,,,
0,"[-0.019771253690123558, -0.02687731385231018, ...",positive,Hierdie film was ongelooflik!,0.857431


In [ ]:
# Afrikaans for :'The movie made me fall asleep, it's awful!'
fitted_pipe.predict('Die film het my aan die slaap laat raak, dit is verskriklik!')

,xx_embed_sentence_labse_embeddings,sentiment,document,sentiment_confidence
origin_index,,,,
0,"[-0.05065516009926796, -0.017065495252609253, ...",negative,"Die film het my aan die slaap laat raak, dit i...",0.999985


# The model understands Vietnamese
![vi](https://www.worldometers.info/img/flags/small/tn_vm-flag.gif)

In [ ]:
# Vietnamese for : 'The movie was painful to watch'
fitted_pipe.predict('Phim đau điếng người xem')


,xx_embed_sentence_labse_embeddings,sentiment,document,sentiment_confidence
origin_index,,,,
0,"[-0.05414637178182602, 0.041687943041324615, -...",negative,Phim đau điếng người xem,1.000000


In [ ]:

# Vietnamese for : 'This was the best movie ever'
fitted_pipe.predict('Đây là bộ phim hay nhất từ ​​trước đến nay')

,xx_embed_sentence_labse_embeddings,sentiment,document,sentiment_confidence
origin_index,,,,
0,"[-0.035795845091342926, -0.005844905506819487,...",neutral,Đây là bộ phim hay nhất từ ​​trước đến nay,0.582394


# The model understands Japanese
![ja](https://www.worldometers.info/img/flags/small/tn_ja-flag.gif)


In [ ]:

# Japanese for : 'This is now my favorite movie!'
fitted_pipe.predict('これが私のお気に入りの映画です！')

,xx_embed_sentence_labse_embeddings,sentiment,document,sentiment_confidence
origin_index,,,,
0,"[-0.00634439941495657, -0.03161682188510895, -...",positive,これが私のお気に入りの映画です！,0.999911


In [ ]:

# Japanese for : 'I would rather kill myself than watch that movie again'
fitted_pipe.predict('その映画をもう一度見るよりも自殺したい')

,xx_embed_sentence_labse_embeddings,sentiment,document,sentiment_confidence
origin_index,,,,
0,"[-0.04823154956102371, -0.03692080080509186, -...",negative,その映画をもう一度見るよりも自殺したい,0.999901


# The model understands Zulu
![zu](https://www.worldometers.info/img/flags/small/tn_sf-flag.gif)

In [ ]:
# Zulu for : 'This movie was crazy good!'
fitted_pipe.predict('Le movie yayihlanya kahle!')


,xx_embed_sentence_labse_embeddings,sentiment,document,sentiment_confidence
origin_index,,,,
0,"[-0.030427439138293266, -0.03501792252063751, ...",positive,Le movie yayihlanya kahle!,0.997382


In [ ]:
# Zulu for : 'This movie was awful!'
fitted_pipe.predict('Le movie yayikhona kabi!')


,xx_embed_sentence_labse_embeddings,sentiment,document,sentiment_confidence
origin_index,,,,
0,"[-0.04461972787976265, -0.04466915875673294, -...",negative,Le movie yayikhona kabi!,0.999999


# The  Model understands Turkish
![tr](https://www.worldometers.info/img/flags/small/tn_tu-flag.gif)

In [ ]:
# Turkish for : 'This movie was awful!'
fitted_pipe.predict('Bu film korkunç!')


,xx_embed_sentence_labse_embeddings,sentiment,document,sentiment_confidence
origin_index,,,,
0,"[-0.028751200065016747, -0.053898535668849945,...",negative,Bu film korkunç!,0.992688


In [ ]:
# Turkish for : 'This movie was great!'
fitted_pipe.predict('Bu film harikaydı!')



,xx_embed_sentence_labse_embeddings,sentiment,document,sentiment_confidence
origin_index,,,,
0,"[-0.024503983557224274, -0.046942222863435745,...",positive,Bu film harikaydı!,0.999670


#  The Model understands Hebrew
![he](https://www.worldometers.info/img/flags/small/tn_sf-flag.gif)

In [ ]:
# Hebrew for : 'This movie was terrible!'
fitted_pipe.predict('הסרט הזה היה נורא!')



,xx_embed_sentence_labse_embeddings,sentiment,document,sentiment_confidence
origin_index,,,,
0,"[-0.039289794862270355, -0.05739438533782959, ...",negative,הסרט הזה היה נורא!,0.999999


In [ ]:
# Hebrew for : 'This movie was great!'
fitted_pipe.predict('הסרט הזה היה נהדר')


,xx_embed_sentence_labse_embeddings,sentiment,document,sentiment_confidence
origin_index,,,,
0,"[-0.03808611258864403, -0.04728467017412186, -...",positive,הסרט הזה היה נהדר,0.927435


# The Model understands Telugu
![te](https://www.worldometers.info/img/flags/small/tn_in-flag.gif)


In [ ]:
# Telugu for : 'This movie was great!'
fitted_pipe.predict('ఈ సినిమా చాలా బాగుంది')

,xx_embed_sentence_labse_embeddings,sentiment,document,sentiment_confidence
origin_index,,,,
0,"[-0.048749782145023346, -0.04968753829598427, ...",positive,ఈ సినిమా చాలా బాగుంది,0.977772


In [ ]:
# Telgu for : 'This movie was awful!'
fitted_pipe.predict('ఈ సినిమా భయంకరంగా ఉంది')

,xx_embed_sentence_labse_embeddings,sentiment,document,sentiment_confidence
origin_index,,,,
0,"[-0.049310874193906784, -0.03890344500541687, ...",negative,ఈ సినిమా భయంకరంగా ఉంది,0.999998


# Model understands Russian
![ru](https://www.worldometers.info/img/flags/small/tn_rs-flag.gif)


In [ ]:
# Russian for : 'This movie was awful!'
fitted_pipe.predict('Этот фильм был ужасен')

,xx_embed_sentence_labse_embeddings,sentiment,document,sentiment_confidence
origin_index,,,,
0,"[-0.04450640082359314, -0.0344516783952713, -0...",negative,Этот фильм был ужасен,0.999997


In [ ]:
# Russian for : 'This movie was great!'
fitted_pipe.predict('Этот фильм был отличным!')

,xx_embed_sentence_labse_embeddings,sentiment,document,sentiment_confidence
origin_index,,,,
0,"[-0.03249521926045418, -0.04056306555867195, -...",positive,Этот фильм был отличным!,0.999809


# Model understands Urdu
![ur](https://www.worldometers.info/img/flags/small/tn_pk-flag.gif)

In [ ]:
# Urdu for : 'This movie was terrible!'
fitted_pipe.predict('یہ فلم خوفناک تھی!')

,xx_embed_sentence_labse_embeddings,sentiment,document,sentiment_confidence
origin_index,,,,
0,"[-0.029061075299978256, -0.04819677397608757, ...",negative,یہ فلم خوفناک تھی!,0.999003


In [ ]:
# Urdu for : 'This movie was great!'
fitted_pipe.predict('یہ فلم زبردست تھی!')

,xx_embed_sentence_labse_embeddings,sentiment,document,sentiment_confidence
origin_index,,,,
0,"[-0.023109590634703636, -0.047220949083566666,...",positive,یہ فلم زبردست تھی!,0.998908


# Model understands Hindi
![hi](https://www.worldometers.info/img/flags/small/tn_in-flag.gif)


In [ ]:
# Hindi for : 'This movie was great!'
fitted_pipe.predict('यह फिल्म बहुत अच्छी थी!')

,xx_embed_sentence_labse_embeddings,sentiment,document,sentiment_confidence
origin_index,,,,
0,"[-0.032199472188949585, -0.05085251107811928, ...",positive,यह फिल्म बहुत अच्छी थी!,0.999490


In [ ]:
# Hindi for : 'This movie was awful!'
fitted_pipe.predict('यह फिल्म भयानक थी!')

,xx_embed_sentence_labse_embeddings,sentiment,document,sentiment_confidence
origin_index,,,,
0,"[-0.034289486706256866, -0.04929641634225845, ...",negative,यह फिल्म भयानक थी!,0.999892


# The model understands Tartar
![tt](https://www.worldometers.info/img/flags/small/tn_rs-flag.gif)

In [ ]:
# Tartar for : 'This movie was terrible!'
fitted_pipe.predict('Бу фильм бик куркыныч иде!')

,xx_embed_sentence_labse_embeddings,sentiment,document,sentiment_confidence
origin_index,,,,
0,"[-0.004161511082202196, -0.04973788559436798, ...",negative,Бу фильм бик куркыныч иде!,0.978463


In [ ]:
# Tartar for : 'This movie was great!'
fitted_pipe.predict('Бу фильм бик яхшы иде!')

,xx_embed_sentence_labse_embeddings,sentiment,document,sentiment_confidence
origin_index,,,,
0,"[-0.013760939240455627, -0.04871012270450592, ...",positive,Бу фильм бик яхшы иде!,0.999856


# The Model understand French
![fr](https://www.worldometers.info/img/flags/small/tn_fr-flag.gif)

In [ ]:
# French for : 'This movie was awful!'
fitted_pipe.predict("ce film était horrible!")

,xx_embed_sentence_labse_embeddings,sentiment,document,sentiment_confidence
origin_index,,,,
0,"[-0.009465612471103668, -0.047107480466365814,...",negative,ce film était horrible!,0.998248


In [ ]:
# French for : 'This movie was great!'
fitted_pipe.predict("Ce film était super!")

,xx_embed_sentence_labse_embeddings,sentiment,document,sentiment_confidence
origin_index,,,,
0,"[-0.013595575466752052, -0.03535566106438637, ...",positive,Ce film était super!,0.999953


# The Model understand Thai
![th](https://www.worldometers.info/img/flags/small/tn_th-flag.gif)

In [ ]:
# Thai for : 'This movie was awful!'
fitted_pipe.predict("หนังเรื่องนี้แย่มาก!")

,xx_embed_sentence_labse_embeddings,sentiment,document,sentiment_confidence
origin_index,,,,
0,"[-0.05294188857078552, -0.05707262083888054, -...",negative,หนังเรื่องนี้แย่มาก!,1.000000


In [ ]:
# Thai for : 'This movie was great!'
fitted_pipe.predict("ภาพยนตร์เรื่องนี้ยอดเยี่ยมมาก!")

,xx_embed_sentence_labse_embeddings,sentiment,document,sentiment_confidence
origin_index,,,,
0,"[-0.03887835890054703, -0.05457264930009842, -...",positive,ภาพยนตร์เรื่องนี้ยอดเยี่ยมมาก!,0.999704


# The Model understand Khmer
![km](https://www.worldometers.info/img/flags/small/tn_cb-flag.gif)

In [ ]:
# Khmer for : 'This movie was awful!'
fitted_pipe.predict("ខ្សែភាពយន្តនេះគួរឱ្យខ្លាចណាស់!")

,xx_embed_sentence_labse_embeddings,sentiment,document,sentiment_confidence
origin_index,,,,
0,"[-0.0342087522149086, -0.056378308683633804, -...",negative,ខ្សែភាពយន្តនេះគួរឱ្យខ្លាចណាស់!,0.999918


In [ ]:
# Khmer for : 'This movie was great!'
fitted_pipe.predict("ខ្សែភាពយន្តនេះអស្ចារ្យណាស់!")

,xx_embed_sentence_labse_embeddings,sentiment,document,sentiment_confidence
origin_index,,,,
0,"[-0.04440384730696678, -0.05587802827358246, -...",positive,ខ្សែភាពយន្តនេះអស្ចារ្យណាស់!,0.999101


# The Model understand Yiddish
![yi](https://www.worldometers.info/img/flags/small/tn_pl-flag.gif)

In [ ]:
# Yiddish for : 'This movie was awful!'
fitted_pipe.predict("דער פֿילם איז געווען שרעקלעך!")

,xx_embed_sentence_labse_embeddings,sentiment,document,sentiment_confidence
origin_index,,,,
0,"[-0.035583771765232086, -0.04656125232577324, ...",negative,דער פֿילם איז געווען שרעקלעך!,0.999930


In [ ]:
# Yiddish for : 'This movie was great!'
fitted_pipe.predict("דער פֿילם איז געווען גרויס!")

,xx_embed_sentence_labse_embeddings,sentiment,document,sentiment_confidence
origin_index,,,,
0,"[-0.032495900988578796, -0.0436769500374794, -...",positive,דער פֿילם איז געווען גרויס!,0.999317


# The Model understand Kygrgyz
![ky](https://www.worldometers.info/img/flags/small/tn_kg-flag.gif)

In [ ]:
# Kygrgyz for : 'This movie was awful!'
fitted_pipe.predict("Бул кино аябай жаман болду!")

,xx_embed_sentence_labse_embeddings,sentiment,document,sentiment_confidence
origin_index,,,,
0,"[-0.035583771765232086, -0.04656125232577324, ...",negative,דער פֿילם איז געווען שרעקלעך!,0.999930


In [ ]:
# Kygrgyz for : 'This movie was great!'
fitted_pipe.predict("Бул кино сонун болду!")

,xx_embed_sentence_labse_embeddings,sentiment,document,sentiment_confidence
origin_index,,,,
0,"[-0.006650532130151987, -0.0518440380692482, -...",positive,Бул кино сонун болду!,0.999928


# The Model understand Tamil
![ta](https://www.worldometers.info/img/flags/small/tn_in-flag.gif)

In [ ]:
# Tamil for : 'This movie was awful!'
fitted_pipe.predict("இந்த படம் மோசமாக இருந்தது!")

,xx_embed_sentence_labse_embeddings,sentiment,document,sentiment_confidence
origin_index,,,,
0,"[-0.04690176621079445, -0.04683435335755348, -...",negative,இந்த படம் மோசமாக இருந்தது!,0.999973


In [ ]:
# Tamil for : 'இந்த படம் நன்றாக இருந்தது!'
fitted_pipe.predict("Бул кино сонун болду!")

,xx_embed_sentence_labse_embeddings,sentiment,document,sentiment_confidence
origin_index,,,,
0,"[-0.006650532130151987, -0.0518440380692482, -...",positive,Бул кино сонун болду!,0.999928
